In [16]:
import numpy as np
import probnum as pn
import linpde_gp
from gp_fvm.plot.dim2 import animate_function, show_anim_notebook, animate_gp

import pykeops
pykeops.verbose = False

pn.config.default_solver_linpde_gp = linpde_gp.solvers.CholeskySolver(dense=True)

# Problem Specification

In [ ]:
a = b = 1
T = 2.
domain = linpde_gp.domains.Box([[0, T], [0, a], [0, b]])
spatial_domain = linpde_gp.domains.Box([[0., a], [0., b]])

c = 1

In [ ]:
from linpde_gp.problems.pde import WaveEquationDirichletProblem

coeffs = np.zeros((3, 3))
coeffs[0, 0] = 1.0
coeffs[1, 1] = -0.5

problem = linpde_gp.problems.pde.WaveEquationDirichletProblem(
    t0=0.,
    T=T,
    spatial_domain=spatial_domain,
    c=c,
    initial_values=linpde_gp.functions.TruncatedSineSeries(
        spatial_domain,
        coefficients=coeffs,
    ),
)

In [ ]:
anim = animate_function(problem.solution, domain, duration=4., use_tqdm=True, zlims=[-0.3, 1.0])
show_anim_notebook(anim)

In [ ]:
from linpde_gp.benchmarking import SolutionErrorEstimator

error_estimator = SolutionErrorEstimator(problem.solution, domain)

# Prior

In [ ]:
lengthscale_t = 0.5
lengthscale_x = a / 3
lengthscale_y = b / 3
output_scale = 1.0
N_ic_xy = 8
N_bc = T * 10

def get_prior(l_t, l_x, l_y, output_scale):
    return pn.randprocs.GaussianProcess(
        mean=linpde_gp.functions.Zero(input_shape=(3,)),
        cov=output_scale**2
        * linpde_gp.randprocs.covfuncs.TensorProduct(
            linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=l_t),
            linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=l_x),
            linpde_gp.randprocs.covfuncs.Matern((), nu=2.5, lengthscales=l_y),
        ),
    )

u_prior = get_prior(lengthscale_t, lengthscale_x, lengthscale_y, output_scale)

# Initial condition

In [ ]:
from linpde_gp.linfuncops.diffops import TimeDerivative

N_ic_xy = 25
N_bc_t = int(T * 10)
N_bc_spatial = 10

def condition_ic(prior):
    X_ic = problem.initial_domain.uniform_grid((N_ic_xy, N_ic_xy))
    Y_ic = problem.initial_condition.values(X_ic[..., 1:])

    u_ic = prior.condition_on_observations(X=X_ic, Y=Y_ic)
    return u_ic.condition_on_observations(X=X_ic, Y=Y_ic, L=TimeDerivative((3,)))

u_ic = condition_ic(u_prior)

# Boundary condition

In [ ]:
def condition_bc(prior):
    boundary_x = domain.uniform_grid((N_bc_t, N_bc_spatial, 2))
    boundary_y = domain.uniform_grid((N_bc_t, 2, N_bc_spatial))

    u_bc = prior.condition_on_observations(X=boundary_x, Y=np.zeros(boundary_x.shape[:-1]))
    return u_bc.condition_on_observations(X=boundary_y, Y=np.zeros(boundary_y.shape[:-1]))

u_ic_bc = condition_bc(u_ic)

#show_anim_notebook(animate_gp(u_ic_bc, domain, with_uncertainty=True))

# PDE observations

In [ ]:
from gp_fvm.finite_volumes import get_grid_from_depth

eval_points = domain.uniform_grid((60, 40, 40))

def condition_fv(prior, depth):
    domains = get_grid_from_depth(domain, depth)
    fv = linpde_gp.linfunctls.FiniteVolumeFunctional(domains, problem.pde.diffop)

    if depth <= 4:
        solver = linpde_gp.solvers.CholeskySolver(dense=True)
    else:
        solver = linpde_gp.solvers.itergp.IterGP_CG_Solver(threshold=1e-2, max_iterations=10000, eval_points=eval_points, num_actions_compressed=400)
    return prior.condition_on_observations(L=fv, Y=np.zeros(domains.shape), solver=solver, fresh_start=depth > 4)

example_depth = 5
u_fv = condition_fv(u_ic_bc, example_depth)

u_fv.representer_weights
anim = animate_gp(u_fv, domain, zlims=[-0.3, 1.0], duration=2., with_uncertainty=True)

In [ ]:
show_anim_notebook(anim)

In [ ]:
error_estimator(u_fv)